In [33]:
# !pip install h3
# !conda install -c conda-forge h3-py
# !pip install --upgrade shapely
# !pip install osmnx --upgrade

In [34]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import h3
import gzip
import shutil
import os
from shapely.geometry import Point
from shapely.geometry import Polygon
import time
import logging
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings("ignore")

### Load the data

In [35]:

start_time = time.time()

file_name = "sr_hex.csv.gz"
var_name = "df_sr_hex"
base_url = "https://cct-ds-code-challenge-input-data.s3.af-south-1.amazonaws.com/"
url = base_url + file_name

print(f"Processing {file_name}...")
!wget -q {url}

# Unzip and load
csv_name = file_name[:-3]  # Remove .gz
with gzip.open(file_name, 'rb') as f_in:
    with open(csv_name, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

os.remove(file_name)  # Optional: clean up compressed file

df = pd.read_csv(csv_name)

#Save for use later
df.to_csv('../data/raw/df_sr_hex.csv', index=False)

print(f"✅ Loaded {file_name} in {time.time() - start_time:.2f} seconds")

Processing sr_hex.csv.gz...


✅ Loaded sr_hex.csv.gz in 17.92 seconds


#### A Quick Look at the Data

In [36]:
df.head().T

,0,1,2,3,4
notification_number,400583534,400555043,400589145,400538915,400568554
reference_number,9109491785.0,9108995239.0,9109614461.0,9108601346.0,NaN
creation_timestamp,2020-10-07 06:55:18+02:00,2020-07-09 16:08:13+02:00,2020-10-27 10:21:59+02:00,2020-03-19 06:36:06+02:00,2020-08-25 09:48:42+02:00
completion_timestamp,2020-10-08 15:36:35+02:00,2020-07-14 14:27:01+02:00,2020-10-28 17:48:15+02:00,2021-03-29 20:34:19+02:00,2020-08-31 08:41:13+02:00
directorate,URBAN MOBILITY,URBAN MOBILITY,URBAN MOBILITY,URBAN MOBILITY,URBAN MOBILITY
department,Roads Infrastructure Management,Roads Infrastructure Management,Roads Infrastructure Management,Roads Infrastructure Management,Roads Infrastructure Management
branch,RIM Area Central,RIM Area East,RIM Area East,RIM Area North,RIM Area South
section,District: Blaauwberg,District : Somerset West,District : Somerset West,District : Bellville,District : Athlone
code_group,TD Customer complaint groups,TD Customer complaint groups,TD Customer complaint groups,TD Customer complaint groups,TD Customer complaint groups
code,Pothole&Defect Road Foot Bic Way/Kerbs,Manhole Cover/Gully Grid,Manhole Cover/Gully Grid,Paint Markings Lines&Signs,Pothole&Defect Road Foot Bic Way/Kerbs


### Get an Overview of the data

In [37]:
print("Dataset Info:")
df.info()
print("\nMissing values:")
print(df.isna().mean().sort_values(ascending=False))

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941634 entries, 0 to 941633
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   notification_number   941634 non-null  int64  
 1   reference_number      592920 non-null  float64
 2   creation_timestamp    941634 non-null  object 
 3   completion_timestamp  929442 non-null  object 
 4   directorate           932199 non-null  object 
 5   department            932180 non-null  object 
 6   branch                913233 non-null  object 
 7   section               848509 non-null  object 
 8   code_group            941634 non-null  object 
 9   code                  941634 non-null  object 
 10  cause_code_group      131117 non-null  object 
 11  cause_code            129669 non-null  object 
 12  official_suburb       729221 non-null  object 
 13  latitude              729270 non-null  float64
 14  longitude             729270 non-null 

#### Thoughts

- many features have missing values not sure if it actually matters
- according to the documentation, missing h3_level8_index values were set to '0'
- there are 22.5% latitude and longitudes that are missing - this means that there are also 22.5% h3_level8_index missing. Quite high but will have to drop these rows as without h3_level8_index the rows are of no value.
- cause_code and cause_code_group both have 86% missing values - won't use these features
- code and code_group have zero missing values.  Let's see the values in these features...





In [38]:

print(f"There are {df.code.nunique()} unique codes and {df.code_group.nunique()} unique code_group values)")
print(f"Unique values in code_group: {df.code_group.unique()}")

There are 492 unique codes and 38 unique code_group values)
Unique values in code_group: ['TD Customer complaint groups' 'ELECTRICITY TECHNICAL COMPLAINTS' 'WATER'
 'WATER AND SANITATION OR METER QUERIES' 'HOUSING - EXISTING'
 'SOLID WASTE' 'REVENUE' 'CITY PARKS' 'HEALTH' 'WATER MANAGEMENT DEVICE'
 'WATER - ALTERNATIVE SYSTEM APPLICATION'
 'SOCIAL DEV. AND EARLY CHILDHOOD DEV.' 'SEWER'
 'ELECTRICITY INTERNAL(REVENUE PROTECTION)'
 'ELECTRICITY FINANCIAL AND METER READING' 'TRAFFIC'
 'ELECTRICITY INTERNAL (GENERAL)' 'TRAFFIC SIGNALS' 'WATER RESTRICTIONS'
 'PLANNING & BUILDING DEVELOPMENT' 'CORPORATE SERVICES CONTACT CENTRE'
 'FIRE' 'WATER  - INFORMAL SETTLEMENTS' 'TRANSPORT' 'PROPERTY MANAGEMENT'
 'WATER POLLUTION CONTROL' 'SEWER - INFORMAL SETTLEMENTS'
 'SPORT AND RECREATION' 'VALUATIONS' 'ACCOUNTS PAYABLE'
 'ELECTRICITY INTERNAL (VENDORS)' 'ENVIRONMENTAL MANAGEMENT'
 'METRO POLICE DEPARTMENT' 'ELECTRICITYINTERNAL (PT&M)'
 'SOLID WASTE SERV. PROVIDER ACCREDITATION'
 'ELECTRICITY INTERNA

In [39]:
df[df.code_group == 'SEWER' ].head(3)

,notification_number,reference_number,creation_timestamp,completion_timestamp,directorate,department,branch,section,code_group,code,cause_code_group,cause_code,official_suburb,latitude,longitude,h3_level8_index
481,1016495961,9.109952e+09,2020-12-26 20:17:23+02:00,2020-12-27 14:48:03+02:00,WATER AND SANITATION,Distribution Services,Reticulation,Reticulation WW Conveyance,SEWER,Sewer: Blocked/Overflow,NaN,NaN,BONTEHEUWEL,-33.949790,18.547810,88ad361033fffff
520,1016481684,9.109929e+09,2020-12-21 08:52:47+02:00,2020-12-21 12:26:02+02:00,WATER AND SANITATION,Distribution Services,Reticulation,Reticulation WW Conveyance,SEWER,Cockroach Spraying,General,Insect Infestation,FRESNAYE,-33.923240,18.386570,88ad361435fffff
603,1015764939,9.108671e+09,2020-04-11 09:54:09+02:00,2020-06-26 12:50:51+02:00,HUMAN SETTLEMENTS,"Home Ownership Transfer, Tenancy Management an...",Upgrades and Maintenance,Area South,SEWER,Sewer: Blocked/Overflow,NaN,NaN,LAVENDER HILL,-34.069183,18.483887,88ad368c21fffff


- Presumably the feature "code_group" is, what is referred to in the introspection challenge as "type"
- Let's drop the rows where h3_level8_index = '0' / missing

In [ ]:
before = df.shape[0]
df = df.loc[df.h3_level8_index != '0']
after = df.shape[0]
dropped = before - after
perc_dropped = dropped / before
print(f"Number of rows dropped: {dropped} Percentage dropped: {perc_dropped:.2}%")

Number of rows dropped: 212364 Percentage dropped: 0.22552711563091393%
